In [16]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"


def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data

def val_missing_data_rbf2_ver2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

class MyDataset2(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def val_missing_data_ori(df):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    #rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    # rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def missing_data_rbf_ori(df, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    #rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    #rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data

def val_missing_data_rbf2(df):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    #rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    # rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model_ori(model, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_ori(df)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))


def eval_model2_ver2_test(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2_ver2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss, x_hat_loss,r_hat_loss, concat_loss, z_hat_loss, c_hat_losss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

def eval_bi_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2_ver2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    loss, x_loss, back_x_loss, loss_c, bi_c_hat = model(dataset)

    Nonscale_imputataion = pd.DataFrame(bi_c_hat[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(bi_c_hat[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(bi_c_hat[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    return Nonscale_imputataion

In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = "cuda" if torch.cuda.is_available() else "cpu"

class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

# Generator 모델
class MGRU4_test(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_test, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.r_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        r_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            gamma_r = self.temp_decay_r(d)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat
            
            r_d = r * gamma_r

            RG = torch.cat([x_c, r_d], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss,r_hat_loss, concat_loss, z_hat_loss, c_hat_loss

In [ ]:
# Generator 모델
class MGRU4_test(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_test, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.r_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        r_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat
            


            RG = torch.cat([x_c, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * concat_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss,r_hat_loss, concat_loss, z_hat_loss, c_hat_loss

In [18]:
def train_MGRU4_test(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_x_hat_loss = 0.0
        batch_r_hat_loss = 0.0
        batch_concat_loss = 0.0
        batch_z_hat_loss = 0.0
        batch_c_hat_loss = 0.0
        for data in dataset:
            data = data.to(device)
            c_hat_list, imputations, x_loss, x_hat_loss,r_hat_loss, concat_loss, z_hat_loss, c_hat_loss  = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
            batch_x_hat_loss += x_hat_loss
            batch_r_hat_loss += r_hat_loss
            batch_concat_loss += concat_loss
            batch_z_hat_loss += z_hat_loss
            batch_c_hat_loss += c_hat_loss

        progress.set_description("loss: {}, x_hat : {}, r_hat : {}, concat_hat : {}, z_hat : {}, c_hat : {}".format(batch_loss, batch_x_hat_loss, batch_r_hat_loss, batch_concat_loss, batch_z_hat_loss, batch_c_hat_loss))

    return x_loss

In [19]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset3 = missing_data_rbf(df, rbfpath, batch_size, 36)

In [21]:
# gamma h 위치 변경
model_ver4_ver3 = MGRU4_test(36, 64)
model_ver4_ver3.to(device)
loss_list = train_MGRU4_test(model_ver4_ver3, 0.001, 1000, dataset3, device)
Nonscale_imputataion2 = eval_model2_ver2_test(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 92.49217224121094, x_hat : 20.183238983154297, r_hat : 31.323871612548828, concat_hat : 1.3989572525024414, z_hat : 22.002296447753906, c_hat : 17.58381462097168: 100%|██████████| 1000/1000 [15:34<00:00,  1.07it/s] 


Scale MAE : tensor(0.1630)
Scale MRE : tensor(0.2354)
Original MAE : 13.149618647917732
Original MRE : 0.18471065614596582
Train MAE : 8.990653772360057
Train MRE : 0.1060535333379353


In [12]:
Nonscale_imputataion2 = eval_model2_ver2_test(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.1583)
Scale MRE : tensor(0.2285)
Original MAE : 12.737328923131093
Original MRE : 0.1789192862494982
Train MAE : 9.038160937304438
Train MRE : 0.10661392670073046


In [22]:
# gamma h 위치 변경
model_ver4_ver3 = MGRU4_test(36, 64)
model_ver4_ver3.to(device)
loss_list = train_MGRU4_test(model_ver4_ver3, 0.001, 1000, dataset3, device)
Nonscale_imputataion2 = eval_model2_ver2_test(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 92.509033203125, x_hat : 20.226402282714844, r_hat : 30.958961486816406, concat_hat : 1.728654146194458, z_hat : 21.991037368774414, c_hat : 17.603975296020508: 100%|██████████| 1000/1000 [15:40<00:00,  1.06it/s]   


Scale MAE : tensor(0.1604)
Scale MRE : tensor(0.2316)
Original MAE : 12.950677604637354
Original MRE : 0.1819161621288712
Train MAE : 8.969126433844405
Train MRE : 0.10579959737612957


In [14]:
# gamma h 위치 변경
model_ver4_ver3 = MGRU4_test(36, 64)
model_ver4_ver3.to(device)
loss_list = train_MGRU4_test(model_ver4_ver3, 0.001, 1000, dataset3, device)
Nonscale_imputataion2 = eval_model2_ver2_test(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 125.8173828125, x_hat : 25.65424919128418, r_hat : 34.749732971191406, concat_hat : 17.139522552490234, z_hat : 25.613670349121094, c_hat : 22.660198211669922:  12%|█▏        | 117/1000 [01:50<13:52,  1.06it/s]    


KeyboardInterrupt: 

In [ ]:
class BiMGRU4_test(nn.Module):
    def __init__(self, input_size, hidden_size, bias = True):
        super(BiMGRU4_test, self).__init__()
        
        self.fmGRU = MGRU4_test(input_size, hidden_size)
        self.bmGRU = MGRU4_test(input_size, hidden_size)
    
    def get_consistency_loss(self, pred_f, pred_b):
        loss = torch.pow(pred_f - pred_b, 2.0).mean()
        return loss
    
    def backdirect_data(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[2])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(2, indices)
    
    def backdirect_imputation(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[1])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(1, indices)

    def forward(self, dataset):
        back_dataset = self.backdirect_data(dataset)

        c_hat_list, x_imputataions, x_loss, x_hat_loss,r_hat_loss, concat_loss, z_hat_loss, c_hat_loss = self.fmGRU(dataset)
        back_c_hat_list, back_x_imputataions, back_x_loss, back_x_hat_loss,back_r_hat_loss ,back_concat_loss, back_z_hat_loss, back_c_hat_loss = self.bmGRU(back_dataset)

        loss_c = self.get_consistency_loss(c_hat_list, self.backdirect_imputation(back_c_hat_list))
        loss = x_loss + back_x_loss + loss_c

        bi_c_hat = (c_hat_list +  self.backdirect_imputation(back_c_hat_list)) / 2

        return  loss, x_loss, back_x_loss, loss_c, bi_c_hat